In [5]:
"""
BERT input:
1 sentence or 2 sentences - 
for 2 sentence: [cls] The man went to the store . [SEP] He bought a dog .
for 1 sentence: [cls] The man went down the street . [SEP]
"""
text = "Here is the sentence I want embeddings for."
marked_text = "[CLS] " + text + " [SEP]"

# tokenize the sentence with the BERT tokenizer
tokenized_text = tokenizer.tokenize(marked_text)

# print out the tokens
print(tokenized_text)

# ['here', 'is', 'the', 'sentence', 'i', 'want', 'em', '##bed', '##ding', '##s', 'for', '.']

# some examples of the tokens contained in the vocab

list(tokenizer.vocab.keys())[5000:5020]


NameError: name 'tokenizer' is not defined

In [ ]:
text = "After stealing money from the bank, the bank robber was seen fishing ont "\
"the Mississippi river bank."
print(text)

# add special tokens
marked_text = "[CLS] "+text+' [SEP]'
tokenized_text = tokenizer.tokenize(marked_text)

# map the token strings to vocab indeces
indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)

# display the tokens with indeces
for tup in zip(tokenized_text, indexed_tokens):
    print(f'{tup[0]:<{12}} {tup[1]}')


In [4]:
# mark each of the 22 tokens as belonging to sentence "1"
segments_ids = [1]*len(tokenized_text)

print(segments_ids)
tokens_tensor = torch.tensor([indexed_tokens])
print(tokens_tensor)
segments_tensors = torch.tensor([segments_ids])
print(segments_tensors)

ImportError: 

IMPORTANT: PLEASE READ THIS FOR ADVICE ON HOW TO SOLVE THIS ISSUE!

Importing the multiarray numpy extension module failed.  Most
likely you are trying to import a failed build of numpy.
Here is how to proceed:
- If you're working with a numpy git repository, try `git clean -xdf`
  (removes all files not under version control) and rebuild numpy.
- If you are simply trying to use the numpy version that you have installed:
  your installation is broken - please reinstall numpy.
- If you have already reinstalled and that did not fix the problem, then:
  1. Check that you are using the Python you expect (you're using /usr/local/opt/python/bin/python3.7),
     and that you have no directories in your PATH or PYTHONPATH that can
     interfere with the Python and numpy versions you're trying to use.
  2. If (1) looks fine, you can open a new issue at
     https://github.com/numpy/numpy/issues.  Please include details on:
     - how you installed Python
     - how you installed

In [ ]:
# load pre-trained model
model = BertModel.from_pretrained('bert-base-uncased',
                                  output_hidden_states = True,# whether the model returns all hidden-states
                                  )
# put the model in "evaluation" mode, meaning feed-forward operation
model.eval()

In [ ]:
# evaluate BERT on example text, fetch the hidden states of the network
# from 12 layers
with torch.no_grad():
    outputs = model(tokens_tensor, segments_tensors)
  # evaluating the model will return a different number of objects based on
  # how it's configured in the 'from_pretrained' call earlier
  # in this case, because we set output_hidden_states=True, the third item will 
  # be the hidden states from all layers
    hidden_states = outputs[2] # the third item

In [ ]:
print("Number of layers:", len(hidden_states))
# output = 13 = initial embeddings + 12 BERT layers
layer_i = 0
print('Number of batches:', len(hidden_states[layer_i]))
batch_i = 0
print('Number of tokens:', len(hidden_states[layer_i][batch_i]))
token_i = 0
print('Number of hidden units:', len(hidden_states[layer_i][batch_i][token_i]))


In [ ]:
# creating word and sentence vectors
# two ways
token_vecs_cat = []
# token_embeddings is a [22 *12 * 768] tensor
# For each token in the sentence...

for token in token_embeddings:
# token is a [12 * 768] tensor
# concatenate the vectors(that is, append them together) from the last four layers
# each layer vector is 768 values, so 'cat_vec' is of length 3072
  cat_vec = torch.cat((token[-1],token[-2],token[-3],token[-4]), dim=0)

  # use 'cat_vec' to represent 'token'
  token_vecs_cat.appebd(cat_vec)
print(f"Shape is {len(token_vecs_cat)}, {len(token_vecs_cat[0])}") # 22 * 768


In [ ]:
# sentence vector
# hidden_states has shape [13 * 1 * 22 * 768]
# 'token_vecs'.shape = [22 * 768]
token_vecs = hidden_states[-2][0]
# hidden_states[-2] this is the second to last hidden layer of each token, size([1,22,768])
# hidden_states[-2][0] eliminate the first batch
token_vecs.shape
sentence_embedding = torch.mean(token_vecs, dim=0)
sentence_embedding.size()

In [14]:
from scipy.spatial.distance import cosine
import torch
cosine(torch.rand(1,2),torch.rand(1,2))

['1049', '1037', '2678', '18657', '1998']